In [ ]:
# =====================================
# 0. 라이브러리
# =====================================
import gradio as gr
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
from openai import OpenAI

# =====================================
# 1. OpenAI 설정 (텍스트 진단용)
# =====================================
OPENAI_API_KEY = "OPENAI_API_KEY"
client = OpenAI(api_key=OPENAI_API_KEY)

FONT_PATH = "/content/ZEN-SERIF-TTF-Regular.ttf"

# =====================================
# 2. 회귀방정식
# =====================================
def predict_view_efficiency(contrast, face_ratio):
    return 4684.06 - (53.78 * contrast) - (40.83 * face_ratio)

# =====================================
# 3. 썸네일 분석 (가상 수치)
# =====================================
def analyze_thumbnail():
    contrast = round(random.uniform(3.3, 4.3), 2)
    face_ratio = round(random.uniform(30, 45), 1)
    score = round(predict_view_efficiency(contrast, face_ratio), 1)
    return contrast, face_ratio, score

# =====================================
# 4. 썸네일 개선 로직
# =====================================
def refine_thumbnail(img):
    # 대비 ↓
    img = ImageEnhance.Contrast(img).enhance(0.75)
    # 채도 ↓
    img = ImageEnhance.Color(img).enhance(0.8)
    # 명도 약간 ↓
    img = ImageEnhance.Brightness(img).enhance(0.95)
    return img

# =====================================
# 6. 진단 리포트 (논리 설명)
# =====================================
def generate_diagnosis(contrast, face_ratio, score):
    text = f"""
썸네일 진단 요약:
- 대비 수치: {contrast}
- 얼굴 면적 비율: {face_ratio}%
- 예상 조회수 효율: {score}%

1. 해당 썸네일은 여러 색상과 명도 요소가 동시에 강조되어 전체 대비 수치가 높은 편입니다.
2. 회귀분석 결과에 따르면 높은 대비는 조회수 효율을 유의미하게 감소시키는 요인으로 작용합니다.
3. 얼굴 면적 비율은 과도하지 않지만, 강한 배경 대비와 결합되어 시각적 집중도를 분산시킬 가능성이 있습니다.
4. 이에 따라 대비와 채도를 완화하고, 시선을 분산시키는 요소를 줄이는 방향으로 개선을 수행했습니다.
"""
    return text

# =====================================
# 7. 클릭 시뮬레이션
# =====================================
def simulate_clicks(old_score, new_score):
    old_clicks = 0
    new_clicks = 0
    for _ in range(100):
        if random.random() < (new_score / (old_score + new_score)):
            new_clicks += 1
        else:
            old_clicks += 1
    return old_clicks, new_clicks

# =====================================
# 8. 전체 파이프라인
# =====================================
def run_pipeline(old_image):
    if old_image is None:
        return "기존 썸네일을 업로드해주세요.", None, None

    contrast, face_ratio, old_score = analyze_thumbnail()

    refined_img = refine_thumbnail(old_image.copy())

    new_contrast = contrast * 0.7
    new_face_ratio = face_ratio * 0.85
    new_score = predict_view_efficiency(new_contrast, new_face_ratio)

    diagnosis = generate_diagnosis(contrast, face_ratio, old_score)

    old_clicks, new_clicks = simulate_clicks(old_score, new_score)

    fig = plt.figure(figsize=(6,4))
    plt.bar(["Original", "Refined"], [old_clicks, new_clicks])
    plt.title("Virtual Viewer Click Simulation (n=100)")
    plt.ylabel("Clicks")

    return diagnosis, fig, refined_img

# =====================================
# 9. Gradio UI
# =====================================
with gr.Blocks(theme=gr.themes.Soft(primary_hue="indigo")) as demo:
    gr.Markdown("# 🎯 Regression-based YouTube Thumbnail Refiner")
    gr.Markdown("기존 썸네일을 회귀분석 인사이트 기반으로 자동 개선합니다.")

    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="기존 썸네일 업로드")
            btn = gr.Button("📈 썸네일 개선")

        with gr.Column():
            output_img = gr.Image(label="개선된 썸네일 (1280×720)")

    out_text = gr.Textbox(label="썸네일 진단 리포트", lines=10)
    out_plot = gr.Plot(label="클릭 시뮬레이션")

    btn.click(
        run_pipeline,
        inputs=input_img,
        outputs=[out_text, out_plot, output_img]
    )

demo.launch(share=True, debug=True)


/tmp/ipython-input-4126354791.py:105: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="indigo")) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c084231387dfccd26b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
